## Get data from Meetup.com

We are going to get data from Meetup's API. You can create custom requests using their API console, then make them via Python by copying the secure link here.

https://secure.meetup.com/meetup_api/console/?path=/2/members

In [3]:
import pandas as pd
import numpy as np
import requests
import os
import json

os.getcwd()

'C:\\Users\\stkba\\OneDrive\\Documents\\GitHub\\stkbailey.github.io\\assets\\pynash_networkx'

#### Get PyNash Members

In [121]:
def get_group_members(group_urlname):
    members = pd.DataFrame()

    for page in np.arange(10):
        q = 'https://api.meetup.com/2/members?format=json&group_urlname={}&page=200&offset={}&key=1eb16676d664fa48314391ae5b6c'.format(group_urlname, page)
        response = requests.get(q).json()
        try:
            tdf = pd.DataFrame.from_dict(response['results']).set_index('id')
            members = members.append(tdf[['name', 'city',  'state', 'hometown', 'joined',  'visited', 'lat', 'lon']])
        except KeyError: 
            continue
        except:
            continue

    members['group_urlname'] = group_urlname
    
    if members.shape[0] > 0:
        return members
    else:
        return None
    
def agg_group_members(list_of_group_urlnames, write_path=None):
    all_members = pd.DataFrame()
    
    for group in list_of_group_urlnames:
        sleep(1)
        try:
            tdf = get_group_members(group)
            all_members = all_members.append(tdf)
        except KeyError as exc:
            print(group, exc)
            continue

    # Write to computer
    if write_path:
        all_members.to_csv(write_path, encoding='utf-8') 
    
    return all_members

In [122]:
all_members = agg_group_members(nash_groups.urlname.tolist(), 'data_allmembers.csv')

From cffi callback <function _verify_callback at 0x000001AF6A78BF28>:
Traceback (most recent call last):
  File "C:\Users\stkba\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 200, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: ("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",)

#### from time import sleep
import json

cols = ('group_id', 'group_name', 'num_members', 'category_id', 
        'category_name', 'organizer_id', 'group_urlname')
nash_groups = pd.DataFrame(columns=cols)

for page in np.arange(10):
    q = 'https://api.meetup.com/find/groups?&sign=true&photo-host=public&location=nashville tn&radius=25&page=200&offset={}&key=1eb16676d664fa48314391ae5b6c'.format(page) 
    response = requests.get(q).json()
    if len(response) > 0:
        try:
            for g in response:
                s = pd.Series((g['id'], g['name'], g['members'], g['category']['id'], 
                               g['category']['name'], g['organizer']['id'], g['urlname']),
                             index=cols)
                nash_groups = nash_groups.append(s, ignore_index=True)
        except: 
            continue
    # Sleep briefly so that API doesn't get overwhelmed
    sleep(0.2)
        

# Write to computer
write_data = True
if write_data == True:
    nash_groups.to_csv('data_groups.csv', index=None) 


#### Get memberships for each member

In [27]:
from time import sleep
import json


edges = pd.DataFrame()
err_ids = []
for pid in members.index:
    r = requests.get('https://api.meetup.com/2/groups?&sign=true&member_id={}&page=200&key=1eb16676d664fa48314391ae5b6c'.format(pid))
    try:
        r = r.json()
        for membership in r['results']:
            edge = pd.Series({'member_id': pid, 
                              'group_id': membership['id'], 
                              'group_name': membership['name']})
            edges = edges.append(edge, ignore_index=True)
    except json.decoder.JSONDecodeError:
        print(pid)
        err_ids.append(pid)
    
    # Sleep briefly so that API doesn't get overwhelmed
    sleep(0.2)
        

# Write to computer
write_data = True
if write_data == True:
    edges.to_csv('data_edges.csv') 
